# Tutorial of DataProcessorUrothelial 

The DataProcessorUrothelial package streamlines the processing of Flatiron Health's advanced urothelial cancer datasets. It provides specialized functions to clean and standardize CSV files containing clinical data (eg., demographics, vitals, labs, medications, ICD codes). Each processing function handles format-specific requirements and common data quality issues, outputting standardized dataframes that can be merged into a single, comprehensive dataset ready for analysis.

In [1]:
# Development setup only 
# These lines are only needed when running this notebook from the repository
import sys
from pathlib import Path
src_path = str(Path('../src').resolve())
sys.path.append(src_path)

## Setup 
To begin using the package, import the required modules and initialize the processor.

In [2]:
from urothelial_processor import DataProcessorUrothelial
from merge_utils import merge_dataframes

import pandas as pd

In [3]:
# Initialize class 
processor = DataProcessorUrothelial()

In [4]:
# Import dataframe with index date of interest for PatientIDs
df = pd.read_csv('../data_uro/Enhanced_AdvUrothelial.csv')

In [5]:
# For our example we'll select 1000 random patients to clean
df = df.sample(n = 1000, random_state = 42)
ids = df.PatientID.to_list()

## Cleaning CSV Files 
Process individual data files using specialized functions. Each function handles data cleaning and standardization specific to the CSV file. 

In [6]:
# Process Enhanced_AdvUrothelial.csv
enhanced_df = processor.process_enhanced_adv(file_path = '../data_uro/Enhanced_AdvUrothelial.csv', 
                                             patient_ids = ids)

2025-02-25 16:16:13,376 - INFO - Successfully read Enhanced_AdvUrothelial.csv file with shape: (13129, 13) and unique PatientIDs: 13129
2025-02-25 16:16:13,376 - INFO - Filtering for 1000 specific PatientIDs
2025-02-25 16:16:13,378 - INFO - Successfully filtered Enhanced_AdvUrothelial.csv file with shape: (1000, 13) and unique PatientIDs: 1000
2025-02-25 16:16:13,386 - INFO - Successfully processed Enhanced_AdvUrothelial.csv file with final shape: (1000, 13) and unique PatientIDs: 1000


In [7]:
# Process Demographics.csv 
demographics_df = processor.process_demographics(file_path = '../data_uro/Demographics.csv',
                                                 index_date_df = df,
                                                 index_date_column = 'AdvancedDiagnosisDate')

2025-02-25 16:16:13,397 - INFO - Successfully read Demographics.csv file with shape: (13129, 6) and unique PatientIDs: 13129
2025-02-25 16:16:13,405 - INFO - Successfully processed Demographics.csv file with final shape: (1000, 6) and unique PatientIDs: 1000


In [8]:
# Process Practice.csv
practice_df = processor.process_practice(file_path = '../data_uro/Practice.csv',
                                         patient_ids = ids)

2025-02-25 16:16:13,417 - INFO - Successfully read Practice.csv file with shape: (14181, 4) and unique PatientIDs: 13129
2025-02-25 16:16:13,418 - INFO - Filtering for 1000 specific PatientIDs
2025-02-25 16:16:13,419 - INFO - Successfully filtered Practice.csv file with shape: (1082, 4) and unique PatientIDs: 1000
2025-02-25 16:16:13,436 - INFO - Successfully processed Practice.csv file with final shape: (1000, 2) and unique PatientIDs: 1000


In [9]:
# Process Enhanced_Mortality_V2.csv and use visit, telemedicine, biomarkers, oral, and progression data to determine censoring date 
mortality_df = processor.process_mortality(file_path = '../data_uro/Enhanced_Mortality_V2.csv',
                                           index_date_df = df, 
                                           index_date_column = 'AdvancedDiagnosisDate',
                                           visit_path = '../data_uro/Visit.csv', 
                                           telemedicine_path = '../data_uro/Telemedicine.csv', 
                                           biomarker_path = '../data_uro/Enhanced_AdvUrothelialBiomarkers.csv', 
                                           oral_path = '../data_uro/Enhanced_AdvUrothelial_Orals.csv',
                                           progression_path = '../data_uro/Enhanced_AdvUrothelial_Progression.csv')

2025-02-25 16:16:13,443 - INFO - Successfully read Enhanced_Mortality_V2.csv file with shape: (9040, 2) and unique PatientIDs: 9040
2025-02-25 16:16:13,452 - INFO - Successfully merged Enhanced_Mortality_V2.csv df with index_date_df resulting in shape: (1000, 3) and unique PatientIDs: 1000
2025-02-25 16:16:13,802 - INFO - The follwing columns ['last_visit_date', 'last_oral_date', 'last_biomarker_date', 'last_progression_date'] are used to calculate the last EHR date
2025-02-25 16:16:13,806 - INFO - Successfully processed Enhanced_Mortality_V2.csv file with final shape: (1000, 3) and unique PatientIDs: 1000. There are 0 out of 1000 patients with missing duration values


In [10]:
# Process Enhanced_AdvUrothelialBiomarkers.csv
biomarkers_df = processor.process_biomarkers(file_path = '../data_uro/Enhanced_AdvUrothelialBiomarkers.csv',
                                             index_date_df = df, 
                                             index_date_column = 'AdvancedDiagnosisDate',
                                             days_before = None, 
                                             days_after = 14)

# Empty values are replaced with "unknown"
biomarkers_df['PDL1_status'] = biomarkers_df['PDL1_status'].fillna('unknown')

biomarkers_df['FGFR_status'] = biomarkers_df['FGFR_status'].cat.add_categories('unknown')
biomarkers_df['FGFR_status'] = biomarkers_df['FGFR_status'].fillna('unknown')

2025-02-25 16:16:13,833 - INFO - Successfully read Enhanced_AdvUrothelialBiomarkers.csv file with shape: (9924, 19) and unique PatientIDs: 4251
2025-02-25 16:16:13,840 - INFO - Successfully merged Enhanced_AdvUrothelialBiomarkers.csv df with index_date_df resulting in shape: (768, 20) and unique PatientIDs: 329
2025-02-25 16:16:13,851 - INFO - Successfully processed Enhanced_AdvUrothelialBiomarkers.csv file with final shape: (1000, 4) and unique PatientIDs: 1000


In [11]:
# Process ECOG.csv
ecog_df = processor.process_ecog(file_path = '../data_uro/ECOG.csv', 
                                 index_date_df = df,
                                 index_date_column = 'AdvancedDiagnosisDate',
                                 days_before = 90,
                                 days_after = 0,
                                 days_before_further = 180)

2025-02-25 16:16:13,914 - INFO - Successfully read ECOG.csv file with shape: (184794, 4) and unique PatientIDs: 9933
2025-02-25 16:16:13,939 - INFO - Successfully merged ECOG.csv df with index_date_df resulting in shape: (14342, 5) and unique PatientIDs: 768
2025-02-25 16:16:13,949 - INFO - Successfully processed ECOG.csv file with final shape: (1000, 3) and unique PatientIDs: 1000


In [12]:
# Process Vitals.csv
vitals_df = processor.process_vitals(file_path = '../data_uro/Vitals.csv',
                                     index_date_df = df,
                                     index_date_column = 'AdvancedDiagnosisDate',
                                     weight_days_before = 90,
                                     days_after = 0,
                                     vital_summary_lookback = 180, 
                                     abnormal_reading_threshold = 1)

2025-02-25 16:16:17,044 - INFO - Successfully read Vitals.csv file with shape: (3604484, 16) and unique PatientIDs: 13109
2025-02-25 16:16:18,425 - INFO - Successfully merged Vitals.csv df with index_date_df resulting in shape: (261747, 17) and unique PatientIDs: 997
2025-02-25 16:16:18,520 - INFO - Successfully processed Vitals.csv file with final shape: (1000, 8) and unique PatientIDs: 1000


In [13]:
# Process Lab.csv
labs_df = processor.process_labs(file_path = '../data_uro/Lab.csv',
                                 index_date_df = df,
                                 index_date_column = 'AdvancedDiagnosisDate',
                                 days_before = 90,
                                 days_after = 0,
                                 summary_lookback = 180)

2025-02-25 16:16:29,531 - INFO - Successfully read Lab.csv file with shape: (9373598, 17) and unique PatientIDs: 12700
2025-02-25 16:16:31,122 - INFO - Successfully merged Lab.csv df with index_date_df resulting in shape: (706793, 18) and unique PatientIDs: 979
2025-02-25 16:16:31,940 - INFO - Successfully processed Lab.csv file with final shape: (1000, 76) and unique PatientIDs: 1000


In [14]:
# Process MedicationAdministration.csv
medications_df = processor.process_medications(file_path = '../data_uro/MedicationAdministration.csv',
                                               index_date_df = df,
                                               index_date_column = 'AdvancedDiagnosisDate',
                                               days_before = 90,
                                               days_after = 0)

2025-02-25 16:16:33,168 - INFO - Successfully read MedicationAdministration.csv file with shape: (997836, 11) and unique PatientIDs: 10983
2025-02-25 16:16:33,297 - INFO - Successfully merged MedicationAdministration.csv df with index_date_df resulting in shape: (71369, 12) and unique PatientIDs: 839
2025-02-25 16:16:33,317 - INFO - Successfully processed MedicationAdministration.csv file with final shape: (1000, 9) and unique PatientIDs: 1000


In [15]:
# Process Diagnsois.csv 
diagnosis_df = processor.process_diagnosis(file_path = '../data_uro/Diagnosis.csv',
                                           index_date_df = df,
                                           index_date_column = 'AdvancedDiagnosisDate',
                                           days_before = None,
                                           days_after = 0)

2025-02-25 16:16:33,733 - INFO - Successfully read Diagnosis.csv file with shape: (625348, 6) and unique PatientIDs: 13129
2025-02-25 16:16:33,799 - INFO - Successfully merged Diagnosis.csv df with index_date_df resulting in shape: (42317, 7) and unique PatientIDs: 1000
2025-02-25 16:16:33,923 - INFO - Successfully processed Diagnosis.csv file with final shape: (1000, 42) and unique PatientIDs: 1000


In [16]:
# Process Insurance.csv 
insurance_df = processor.process_insurance(file_path = '../data_uro/Insurance.csv',
                                           index_date_df = df,
                                           index_date_column = 'AdvancedDiagnosisDate',
                                           days_before = None,
                                           days_after = 0,
                                           missing_date_strategy = 'liberal')

2025-02-25 16:16:33,999 - INFO - Successfully read Insurance.csv file with shape: (53709, 14) and unique PatientIDs: 12391
2025-02-25 16:16:34,025 - INFO - Successfully merged Insurance.csv df with index_date_df resulting in shape: (3861, 15) and unique PatientIDs: 943
2025-02-25 16:16:34,036 - INFO - Successfully processed Insurance.csv file with final shape: (1000, 5) and unique PatientIDs: 1000


## Merge Processed Dataframes
Merge the processed dataframes into a single analysis-ready dataset

In [17]:
merged_data = merge_dataframes(enhanced_df, 
                               demographics_df, 
                               practice_df, 
                               mortality_df, 
                               biomarkers_df, 
                               ecog_df, 
                               vitals_df,
                               labs_df,
                               medications_df, 
                               diagnosis_df, 
                               insurance_df)

2025-02-25 16:16:34,039 - INFO - Anticipated number of merges: 10
2025-02-25 16:16:34,040 - INFO - Anticipated number of columns in final dataframe presuming all columns are unique except for PatientID: 161
2025-02-25 16:16:34,041 - INFO - Dataset 1 shape: (1000, 13), unique PatientIDs: 1000
2025-02-25 16:16:34,041 - INFO - Dataset 2 shape: (1000, 6), unique PatientIDs: 1000
2025-02-25 16:16:34,042 - INFO - Dataset 3 shape: (1000, 2), unique PatientIDs: 1000
2025-02-25 16:16:34,043 - INFO - Dataset 4 shape: (1000, 3), unique PatientIDs: 1000
2025-02-25 16:16:34,043 - INFO - Dataset 5 shape: (1000, 4), unique PatientIDs: 1000
2025-02-25 16:16:34,044 - INFO - Dataset 6 shape: (1000, 3), unique PatientIDs: 1000
2025-02-25 16:16:34,044 - INFO - Dataset 7 shape: (1000, 8), unique PatientIDs: 1000
2025-02-25 16:16:34,044 - INFO - Dataset 8 shape: (1000, 76), unique PatientIDs: 1000
2025-02-25 16:16:34,045 - INFO - Dataset 9 shape: (1000, 9), unique PatientIDs: 1000
2025-02-25 16:16:34,045 - 

In [18]:
merged_data.columns.to_list()

['PatientID',
 'PrimarySite',
 'DiseaseGrade',
 'NStage',
 'SmokingStatus',
 'Surgery',
 'GroupStage_mod',
 'TStage_mod',
 'MStage_mod',
 'SurgeryType_mod',
 'days_diagnosis_to_adv',
 'adv_diagnosis_year',
 'days_diagnosis_to_surgery',
 'Gender',
 'Race',
 'Ethnicity',
 'age',
 'region',
 'PracticeType_mod',
 'event',
 'duration',
 'PDL1_status',
 'PDL1_percent_staining',
 'FGFR_status',
 'ecog_index',
 'ecog_newly_gte2',
 'weight',
 'bmi',
 'percent_change_weight',
 'hypotension',
 'tachycardia',
 'fevers',
 'hypoxemia',
 'albumin',
 'alp',
 'alt',
 'ast',
 'bicarb',
 'bun',
 'calcium',
 'chloride',
 'creatinine',
 'hemoglobin',
 'platelet',
 'potassium',
 'sodium',
 'total_bilirubin',
 'wbc',
 'albumin_max',
 'alp_max',
 'alt_max',
 'ast_max',
 'bicarb_max',
 'bun_max',
 'calcium_max',
 'chloride_max',
 'creatinine_max',
 'hemoglobin_max',
 'platelet_max',
 'potassium_max',
 'sodium_max',
 'total_bilirubin_max',
 'wbc_max',
 'albumin_min',
 'alp_min',
 'alt_min',
 'ast_min',
 'bicarb

In [19]:
for col, dtype in merged_data.dtypes.items():
    print(f"{col}: {dtype}")

PatientID: object
PrimarySite: category
DiseaseGrade: category
NStage: category
SmokingStatus: category
Surgery: int64
GroupStage_mod: category
TStage_mod: category
MStage_mod: category
SurgeryType_mod: category
days_diagnosis_to_adv: float64
adv_diagnosis_year: category
days_diagnosis_to_surgery: float64
Gender: category
Race: category
Ethnicity: category
age: Int64
region: category
PracticeType_mod: category
event: Int64
duration: float64
PDL1_status: category
PDL1_percent_staining: category
FGFR_status: category
ecog_index: category
ecog_newly_gte2: Int64
weight: float64
bmi: float64
percent_change_weight: float64
hypotension: Int64
tachycardia: Int64
fevers: Int64
hypoxemia: Int64
albumin: float64
alp: float64
alt: float64
ast: float64
bicarb: float64
bun: float64
calcium: float64
chloride: float64
creatinine: float64
hemoglobin: float64
platelet: float64
potassium: float64
sodium: float64
total_bilirubin: float64
wbc: float64
albumin_max: float64
alp_max: float64
alt_max: float64
